# API

In this first part, we will extract all the Glassdoor reviews about Typeform. With these, you will be able to have a proper overview of the current status of the company from an employee point of view, and make a good judgement call on whether this is the company you would like to work at.

## Libraries

In [1]:
import json
import requests
import pandas as pd
from pandas.io.json import json_normalize
import datetime as dt

## Obtaining the link and Reading the data

In [2]:
token = ''

In [3]:
url = 'https://wextractor.com/api/v1/reviews/glassdoor?id=991912&language=en&auth_token=' + token

In [4]:
response = requests.get(url).json()

In [5]:
glassdoor = pd.json_normalize(response['reviews'])
glassdoor.head()

,url,rating,title,pros,cons,advice,reviewer,datetime,language,id
0,https://www.glassdoor.com/Reviews/Employee-Rev...,3,Good product bad pay,The product is amazing and so are the people!,Not the greatest base pay and no real leadership,None,Account Executive,2021-03-13T16:30:31.267,en,43952934
1,https://www.glassdoor.com/Reviews/Employee-Rev...,5,I love Typeform!,"Good CEO, salary &amp; time","Coffee, lunch and snacks and time off",None,Growth Manager,2021-03-04T17:59:06.410,en,43500767
2,https://www.glassdoor.com/Reviews/Employee-Rev...,5,Good,This is no pros and i like it,Highly efficient and care for employes,None,Data Entry,2021-03-02T19:37:32.570,en,43372871
3,https://www.glassdoor.com/Reviews/Employee-Rev...,5,Best onboarding ever. 🤩,"Professional, driven and empathetic humans who...",Timezone workarounds due to international hybr...,"One day, it would be absolutely amazing to acq...",None,2021-02-11T21:37:06.797,en,42376252
4,https://www.glassdoor.com/Reviews/Employee-Rev...,5,Feedback,Smooth interview process. Friendly and Authent...,So far no need for constructive feedback :),None,People Ops,2021-02-10T06:24:51.197,en,42259875


### Total Pages

In [6]:
number_reviews = response['totals']['review_count']
pages = int(round(number_reviews / len(glassdoor)))
print(f'There are {pages} pages.')

There are 11 pages.


### Total Reviews

In [7]:
total_reviews = []
offset = 0

for page in range(pages):
    total_response = requests.get(url + '&offset='+str(offset)).json()
    offset = offset + 10
    total_reviews.append(pd.json_normalize(total_response['reviews']))

In [8]:
glassdoor = pd.concat(total_reviews)
glassdoor.reset_index(inplace=True)
glassdoor.drop(columns=['index', 'language', 'id'], inplace=True)
glassdoor.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 108 entries, 0 to 107
Data columns (total 8 columns):
 #   Column    Non-Null Count  Dtype 
---  ------    --------------  ----- 
 0   url       108 non-null    object
 1   rating    108 non-null    object
 2   title     108 non-null    object
 3   pros      108 non-null    object
 4   cons      108 non-null    object
 5   advice    65 non-null     object
 6   reviewer  73 non-null     object
 7   datetime  108 non-null    object
dtypes: object(8)
memory usage: 6.9+ KB


In [9]:
glassdoor['date'] = pd.to_datetime(glassdoor['datetime']).dt.strftime('%Y-%m-%d')

In [10]:
glassdoor['rating'] = pd.to_numeric(glassdoor['rating'])

In [11]:
glassdoor.columns.to_list()

['url',
 'rating',
 'title',
 'pros',
 'cons',
 'advice',
 'reviewer',
 'datetime',
 'date']

In [12]:
cols = ['date', 'rating', 'title', 'pros', 'cons', 'advice', 'reviewer', 'url']
glassdoor = glassdoor[cols]

In [13]:
glassdoor.fillna('None', inplace=True)

In [14]:
# Sort by date (newest to oldest)
glassdoor.sort_values(by='date', ascending=False, inplace=True)

In [15]:
glassdoor.head()

,date,rating,title,pros,cons,advice,reviewer,url
0,2021-03-13,3,Good product bad pay,The product is amazing and so are the people!,Not the greatest base pay and no real leadership,None,Account Executive,https://www.glassdoor.com/Reviews/Employee-Rev...
1,2021-03-04,5,I love Typeform!,"Good CEO, salary &amp; time","Coffee, lunch and snacks and time off",None,Growth Manager,https://www.glassdoor.com/Reviews/Employee-Rev...
2,2021-03-02,5,Good,This is no pros and i like it,Highly efficient and care for employes,None,Data Entry,https://www.glassdoor.com/Reviews/Employee-Rev...
3,2021-02-11,5,Best onboarding ever. 🤩,"Professional, driven and empathetic humans who...",Timezone workarounds due to international hybr...,"One day, it would be absolutely amazing to acq...",None,https://www.glassdoor.com/Reviews/Employee-Rev...
4,2021-02-10,5,Feedback,Smooth interview process. Friendly and Authent...,So far no need for constructive feedback :),None,People Ops,https://www.glassdoor.com/Reviews/Employee-Rev...


## Ratings

### Overall Rating

In [16]:
overall_rating = response['totals']['overall_rating']
print(f"Typeform's overall rating is {overall_rating}.")

Typeform's overall rating is 4.2.


### Ratings Percentage

In [17]:
ratings = glassdoor['rating'].value_counts()
ratings_df = pd.DataFrame(ratings)
rating_percentage = (ratings_df['rating'] / number_reviews) * 100
ratings_df['percentage'] = round(rating_percentage, 2)
ratings_df

,rating,percentage
5,59,54.13
4,20,18.35
1,12,11.01
2,11,10.09
3,6,5.50


### Total Ratings

In [18]:
total_ratings_dict = [{
    'overall_rating': overall_rating,
    'culture_and_values': response['totals']['culture_and_values_rating'],
    'diversity_and_inclusion': response['totals']['diversity_and_inclusion_rating'],
    'work_life_balance': response['totals']['work_life_balance_rating'],
    'senior_management': response['totals']['senior_management_rating'],
    'compensation_and_benefits': response['totals']['compensation_and_benefits_rating'],
    'career_opportunities': response['totals']['career_opportunities_rating']
}]

In [19]:
total_ratings = pd.DataFrame(total_ratings_dict)

In [20]:
cols = ['overall_rating', 'culture_and_values', 'diversity_and_inclusion', 'work_life_balance', 'senior_management', 'compensation_and_benefits', 'career_opportunities']

In [21]:
total_ratings[cols] = total_ratings[cols].apply(pd.to_numeric)

In [22]:
total_ratings.melt()

,variable,value
0,overall_rating,4.2
1,culture_and_values,4.1
2,diversity_and_inclusion,3.8
3,work_life_balance,4.3
4,senior_management,3.6
5,compensation_and_benefits,4.4
6,career_opportunities,3.8


## Export

In [23]:
glassdoor.to_csv('1-reviews.csv', sep=',', index=False)